In [ ]:
#needed to make web requests
import requests
#convert the response as a strcuctured json
import json

import pandas as pd
import numpy as np
#parse the datetimes we get from NOAA
from datetime import datetime

In [ ]:
#add the access token you got from NOAA
Token = 'JvgZzITnvipZbRytfbTgJVeCmCvLwhWu'

#Long Beach Airport station
station_id = 'GHCND:USW00023129'

In [ ]:

#initialize lists to store data
dates_temp = []
dates_prcp = []
temps = []
prcp = []

#for each year from 2015-2019 ...
for year in range(2015, 2020):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]

In [ ]:
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&locationcategoryid=ST&Locations=Alabama&startdate=2020-01-01&enddate=2020-01-01', headers={'token':Token})

In [ ]:
d = json.loads(r.text)

In [ ]:
len(d['results'])

In [ ]:
def get_temperature():
    baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'
    params_diction = {}
    params_diction['datasetid'] = 'GHCND'
    params_diction['datatypeid'] = 'TAVG'
    params_diction['limit'] = 1000
    params_diction['locationcategoryid'] = 'ST'
    params_diction['startdate'] = '2020-01-01'
    params_diction['enddate'] = '2021-05-07'
    response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
    temperature_dict = response.json()
    return temperature_dict

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations'
params_diction = {}
# params_diction['datasetid'] = 'GHCND'
# params_diction['datatypeid'] = 'TAVG'
params_diction['limit'] = 1000
# params_diction['locationcategoryid'] = 'ST'
# params_diction['stationid'] = 'GHCND:USW00023129'
# params_diction['startdate'] = '2020-01-01'
# params_diction['enddate'] = '2021-05-07'
params_diction['offset'] = 1000
response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})

In [ ]:
response.url

In [ ]:
np.arange(0, 38862, 1000)

In [ ]:
def Merge(dict1, dict2):
    return(dict2.update(dict1))

In [ ]:
demo_dict = {}
offset = 0
params_diction = {}
params_diction['offset'] = offset
response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
current_dict = json.loads(response.text)

In [ ]:
d = {}
for item in current_dict['results']:
    d[item['name']] = item['id']

In [ ]:
d

In [ ]:
#initialize dataframe
df_temp = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime and convert temperature from tenths of Celsius to Fahrenheit)
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]

# Get all locationid

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations'
all_locations_dict = {}
for offset in np.arange(0, 38862, 1000):
    params_diction = {}
    params_diction['offset'] = offset
    params_diction['limit'] = 1000
    response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
    current_dict = json.loads(response.text)
    for item in current_dict['results']:
        all_locations_dict[item['id']] = item['name']

In [ ]:
len(all_locations_dict)

In [ ]:
df = pd.DataFrame.from_dict([all_locations_dict])

In [ ]:
df = pd.melt(df)

In [ ]:
df.to_excel('all_locations.xlsx')

# Get number of records per each locationid

In [ ]:
mylist = ['01', '02', '03', '04', '05', '06', '07', '08', '09']
from_ten = list(np.arange(10, 80, 1))
from_ten = [str(x) for x in from_ten]
mylist.extend(from_ten)
mylist = [str(x) for x in mylist]
len(mylist)

In [ ]:
mylist.remove('03')
mylist.remove('07')
mylist.remove('11')
mylist.remove('14')
mylist.remove('43')
mylist.remove('52')
mylist.remove('57')
mylist.remove('58')
mylist.remove('59')
mylist.remove('61')
mylist.remove('62')
mylist.remove('63')
mylist.remove('64')
mylist.remove('65')
mylist.remove('67')
mylist.remove('68')
mylist.remove('69')
mylist.remove('70')
mylist.remove('71')
mylist.remove('72')
mylist.remove('73')
mylist.remove('74')
mylist.remove('75')
mylist.remove('76')
mylist.remove('77')
mylist.remove('78')


len(mylist)

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'
records_dict = {}
for n in mylist:
    locationid = f'FIPS:{n}'
    print(locationid)
    params_diction = {}
    params_diction['datasetid'] = 'GHCND'
    params_diction['limit'] = 1000
    params_diction['datatypeid'] = 'TAVG'

    params_diction['locationid']=locationid

    params_diction['startdate'] = '2020-01-01'
    params_diction['enddate'] = '2020-12-31'
    response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
    current_dict = json.loads(response.text)
    number_of_records = current_dict['metadata']['resultset']['count']
    records_dict[locationid] = number_of_records

In [ ]:
df_records_dict = pd.DataFrame.from_dict([records_dict])

In [ ]:
df_records_dict = pd.melt(df_records_dict)
len(df_records_dict)

In [ ]:
df_records_dict.to_csv('records_per_loc.csv')

In [ ]:
df.head()

In [ ]:
df.columns = ['locationid', 'location']
df.head()

In [ ]:
df_records_dict.columns = ['locationid', 'location']
df_records_dict.head()

In [ ]:
location_df = pd.merge(df_records_dict, df, how='inner', on='locationid')
location_df.head()

In [ ]:
location_df.columns = ['locationid', 'records_count', 'state']
location_df.head()

In [ ]:
for i in range(len(location_df)):
    print(location_df.iloc[i]['records_count'])

In [379]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'

i = 0
offset = 1000

state = location_df.iloc[i]['state']
records_count = location_df.iloc[i]['records_count']
locationid = location_df.iloc[i]['locationid']

df_output = pd.DataFrame()

for offset in np.arange(1, records_count, 1000):
    print(state)
    params_diction = {}
    params_diction['datasetid'] = 'GHCND'
    params_diction['limit'] = 1000
    params_diction['datatypeid'] = 'TAVG'

    params_diction['locationid']=locationid

    params_diction['offset'] = offset

    params_diction['startdate'] = '2020-01-01'
    params_diction['enddate'] = '2020-12-31'

    response = requests.request('GET', baseurl, params=params_diction, headers={'token':Token})
    current_dict = json.loads(response.text)

    for item in current_dict['results']:
        df_temporary = pd.DataFrame([item])
        df_output = df_output.append(df_temporary)

Alabama
Alabama
Alabama
Alabama
Alabama


In [380]:
current_dict['metadata']

{'resultset': {'offset': 4001, 'count': 4758, 'limit': 1000}}

In [381]:
len(df_output)

4758

In [382]:
df_output.head()

,date,datatype,station,attributes,value
0,2020-01-01T00:00:00,TAVG,GHCND:USR0000ABAN,",,U,",72
0,2020-01-01T00:00:00,TAVG,GHCND:USR0000ABNS,",,U,",135
0,2020-01-01T00:00:00,TAVG,GHCND:USR0000ALIR,",,U,",38
0,2020-01-01T00:00:00,TAVG,GHCND:USR0000AOKM,",,U,",66
0,2020-01-01T00:00:00,TAVG,GHCND:USR0000AOPE,",,U,",102


In [383]:
df_output.tail()

,date,datatype,station,attributes,value
0,2020-12-31T00:00:00,TAVG,GHCND:USW00013829,"H,,S,",133
0,2020-12-31T00:00:00,TAVG,GHCND:USW00013876,"H,,S,",155
0,2020-12-31T00:00:00,TAVG,GHCND:USW00013894,"H,,S,",184
0,2020-12-31T00:00:00,TAVG,GHCND:USW00013895,"H,,S,",164
0,2020-12-31T00:00:00,TAVG,GHCND:USW00053864,"H,,S,",144


In [ ]:
print(state,records_count,locationid)

In [366]:
df_output = pd.DataFrame()
df_output

""


In [367]:

for item in current_dict['results']:
    df_temporary = pd.DataFrame([item])
    df_output = df_output.append(df_temporary)

In [ ]:
# for offset in np.arange(0, 38862, 1000):
# number_of_records = current_dict['metadata']['resultset']['count']
# records_dict[locationid] = number_of_records


# for item in current_dict['results']:
#         all_locations_dict[item['id']] = item['name']